# 02 – Hex Lattice Baseline Experiments

This notebook explores a **hexagonal lattice baseline** for the
Santa 2025 – Christmas Tree Packing Challenge.

Goals:
- Define the tree polygon and its circumscribed radius.
- Construct a hexagonal grid of candidate centers.
- Build simple non-overlapping layouts for a given `n` by taking
  the `n` closest centers to the origin.
- Visualize the layouts and approximate their bounding squares.

This is a sandbox for developing and understanding a unique baseline
packing strategy before moving the logic into `src/santa2025/packers/hex_lattice.py`.

In [ ]:
import math
from dataclasses import dataclass
from typing import List, Tuple

import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

%matplotlib inline
plt.rcParams['figure.dpi'] = 120


In [ ]:
# Tree polygon in local coordinates
# Origin (0, 0) = center of the top of the trunk

TREE_TEMPLATE_VERTS = np.array([
    [0.0, 0.8],          # Tip of the tree
    [0.25 / 2, 0.5],     # Right top tier
    [0.25 / 4, 0.5],
    [0.4 / 2, 0.25],     # Right middle tier
    [0.4 / 4, 0.25],
    [0.7 / 2, 0.0],      # Right bottom tier
    [0.15 / 2, 0.0],     # Right trunk top
    [0.15 / 2, -0.2],    # Right trunk bottom
    [-0.15 / 2, -0.2],   # Left trunk bottom
    [-0.15 / 2, 0.0],    # Left trunk top
    [-0.7 / 2, 0.0],     # Left bottom tier
    [-0.4 / 4, 0.25],    # Left middle tier
    [-0.4 / 2, 0.25],
    [-0.25 / 4, 0.5],    # Left top tier
    [-0.25 / 2, 0.5],
], dtype=float)

TREE_RADIUS = float(np.linalg.norm(TREE_TEMPLATE_VERTS, axis=1).max())
TREE_RADIUS


In [ ]:
def make_tree_polygon(x: float, y: float, angle_deg: float) -> Polygon:
    """Build a tree polygon at (x, y) with rotation angle_deg."""
    theta = math.radians(angle_deg)
    c, s = math.cos(theta), math.sin(theta)
    rot = np.array([[c, -s], [s, c]], dtype=float)
    pts = TREE_TEMPLATE_VERTS @ rot.T
    pts[:, 0] += x
    pts[:, 1] += y
    return Polygon(pts)

@dataclass
class TreePose:
    x: float
    y: float
    angle: float
    poly: Polygon

    @classmethod
    def from_params(cls, x: float, y: float, angle: float) -> "TreePose":
        return cls(x=x, y=y, angle=angle, poly=make_tree_polygon(x, y, angle))


In [ ]:
def generate_hex_centers(num_points: int, radius: float = TREE_RADIUS) -> List[Tuple[float, float, int, int]]:
    """Generate a hexagonal lattice of centers around the origin.
    Returns (x, y, row, col) sorted by distance to origin.
    """
    dx = 2.0 * radius
    dy = math.sqrt(3.0) * radius
    max_ring = 20  # generous patch size
    centers = []
    for row in range(-max_ring, max_ring + 1):
        offset = 0.5 * dx if (row & 1) else 0.0
        for col in range(-max_ring, max_ring + 1):
            x = col * dx + offset
            y = row * dy
            centers.append((x, y, row, col))
    centers.sort(key=lambda c: c[0] * c[0] + c[1] * c[1])
    return centers[:num_points]

HEX_CENTERS = generate_hex_centers(200)
len(HEX_CENTERS), HEX_CENTERS[0]


In [ ]:
def initial_hex_layout_for_n(n: int) -> List[TreePose]:
    """Take the first n hex centers and assign a simple orientation pattern."""
    poses: List[TreePose] = []
    for k in range(n):
        x, y, row, col = HEX_CENTERS[k]
        # Simple pattern: flip angle by row parity
        angle = 0.0 if (row % 2 == 0) else 180.0
        poses.append(TreePose.from_params(x, y, angle))
    return poses

# Quick smoke test for a small n
poses_5 = initial_hex_layout_for_n(5)
len(poses_5), poses_5[0]


In [ ]:
def has_any_collision(poses: List[TreePose]) -> bool:
    polys = [p.poly for p in poses]
    index = STRtree(polys)
    for i, poly in enumerate(polys):
        candidates = index.query(poly)
        for j in candidates:
            if i == j:
                continue
            if poly.intersects(polys[j]) and not poly.touches(polys[j]):
                return True
    return False

print('n=5 collision?', has_any_collision(poses_5))


In [ ]:
from shapely.ops import unary_union

def bounding_square_side(poses: List[TreePose]) -> float:
    union = unary_union([p.poly for p in poses])
    minx, miny, maxx, maxy = union.bounds
    width, height = maxx - minx, maxy - miny
    return max(width, height)

def plot_layout(poses: List[TreePose], title: str = "", figsize=(6, 6)):
    union = unary_union([p.poly for p in poses])
    minx, miny, maxx, maxy = union.bounds
    side = max(maxx - minx, maxy - miny)
    fig, ax = plt.subplots(figsize=figsize)
    for p in poses:
        xs, ys = p.poly.exterior.xy
        ax.fill(xs, ys, alpha=0.4)
        ax.plot(xs, ys, linewidth=0.8)
    # bounding square
    ax.add_patch(plt.Rectangle((minx, miny), side, side,
                               fill=False, edgecolor='red', linestyle='--', linewidth=2))
    pad = 0.5
    ax.set_xlim(minx - pad, minx + side + pad)
    ax.set_ylim(miny - pad, miny + side + pad)
    ax.set_aspect('equal', adjustable='box')
    ax.axis('off')
    ax.set_title(f"{title} (side≈{side:.4f})")
    plt.show()

# Example for n = 10
poses_10 = initial_hex_layout_for_n(10)
print('n=10 collision?', has_any_collision(poses_10))
plot_layout(poses_10, title='Hex baseline, n=10')


In [ ]:
def summarize_hex_baseline(ns):
    rows = []
    for n in ns:
        poses = initial_hex_layout_for_n(n)
        side = bounding_square_side(poses)
        rows.append((n, side, side * side / n))
    return rows

ns = [1, 2, 3, 5, 10, 20, 50, 100, 200]
rows = summarize_hex_baseline(ns)
for n, side, score in rows:
    print(f"n={n:3d}  side≈{side:8.4f}  contrib≈{score:10.6f}")


In [ ]:
for n in [20, 50, 100, 200]:
    poses = initial_hex_layout_for_n(n)
    print(f"n={n}, collision? {has_any_collision(poses)}")
    plot_layout(poses, title=f"Hex baseline, n={n}", figsize=(6, 6))
